# Gates on different platforms


While the spin is the common basis of quantum information processing, we can also store the information in moving particles, which might interact. These different platforms will have quite different natural gates as we will see.


Let us start out with the simplest situation of a single qubit as it sets the reference.  We will then move on a to a single hopping particle. Finally we can discuss multi-particle physics if time permits.


# Qubits

## A single qubit

Let us start out with the example of a single spin. Its circuit then typically looks as follows:

![quantum circuit](SingleSpin.svg)

Its Hilbert space is of dimension two, with the computational basis being the states $\{\uparrow, \downarrow\}$. We then have the three Pauli matrices that generate gates on this Hilbert space.  

These gates are generated by the evolution of the spin under the following Hamiltonian:

\begin{align}
\hat{H} = \frac{\Delta}{2} \hat{\sigma}_z +\frac{\Omega_x}{2} \hat{\sigma}_x + \frac{\Omega_y}{2}\hat{\sigma}_y
\end{align}

The parameters $\Delta, \Omega_x, \Omega_y$ are adjusted on the hardware to implement the relevant gate. The gate arises then from the unitary evolution under the Hamiltonian for a time $t$:

\begin{align}
\hat{U} = e^{i\hat{H}t}
\end{align}

Let us now implement the Hamiltonian of this simple example.

In [21]:
import numpy as np

set up the Pauli matrices and test that they properly commute

In [22]:
paulix = np.matrix([[0,1], [1,0]])
pauliy = np.matrix([[0,-1j], [1j,0]])
pauliz = np.matrix([[1,0], [0,-1]])

assert (paulix.dot(pauliy)-pauliy.dot(paulix) == 2j*pauliz).all(), 'Do not commute properly'
assert (pauliy.dot(pauliz)-pauliz.dot(pauliy) == 2j*paulix).all(), 'Do not commute properly'
assert (pauliz.dot(paulix)-paulix.dot(pauliz) == 2j*pauliy).all(), 'Do not commute properly'

time to puzzle together the Hamiltonian

In [23]:
def Ham_sq(delta, omegax, omegay):
    '''
    The Hamiltonian matrix of a single qubit
    '''
    return delta/2*pauliz+omegax/2*paulix+omegay/2*pauliy

and put together the resulting unitary gate

In [24]:
from scipy.linalg import expm

delta = 0; omegax = 1; omegay = 0;
Ham = Ham_sq(delta, omegax, omegay)
t = np.pi;
U = expm(1j*Ham*t)
print(U)

[[0.+0.j 0.+1.j]
 [0.+1.j 0.+0.j]]


and we can end up with the example where we apply this gate to flip the qubit from the zero state into the down state.

In [25]:
psi0 = np.matrix([1,0])
psif = U.dot(psi0.T)
print(psif)

[[0.+0.j]
 [0.+1.j]]


## Two coupled qubits

The situation is substantially more interesting if we employ two spins. As they might interact with each other and have single qubit gates

![quantum circuit](TwoSpins.svg)

These gates are generated by the evolution of the spin under the following Hamiltonian:

\begin{align}
\hat{H} = \sum_{i=1,2} \left[\frac{\Delta_i}{2} \hat{\sigma}^i_z +\frac{\Omega_{x,i}}{2} \hat{\sigma}^i_x + \frac{\Omega_{y,i}}{2}\hat{\sigma}^i_y\right] + \Omega_c \hat{\sigma}^1_z \hat{\sigma}^2_z
\end{align}

The parameters $\Delta, \Omega_x, \Omega_y, \Omega_c$ are adjusted on the hardware to implement the relevant gate. The gate arises then from the unitary evolution under the Hamiltonian for a time $t$:

\begin{align}
\hat{U} = e^{i\hat{H}t}
\end{align}


Applying a non-zero $\Omega_c$ leads typically to entanglement between the qubits.

# Moving particles 

Let us now assume that our basic unit of information is not a single spin, but some moving particles, which might sit on some site $i$. The computational basis will then be the occupation of the site, noted by $n_i$.  We can once again draw the typical circuit diagram of such a hardware.

![two hopping particles](TwoSites.svg)

As we can see it has an extremely similiar structure to the discussion above. However, we will discuss in the next step that this hardware is implementing quite different sets of operators.


This system is mathematically described in second quantization with the raising and lowering operators $\hat{a}_i^\dagger$ and $\hat{a}_i$. 

If the moving particle is a _boson_, then raising operators action is given by:

\begin{align}
\hat{a}_i^\dagger |\cdots, n_i, \cdots\rangle = \sqrt{n_i+1}|\cdots, n_i+1, \cdots\rangle 
\end{align}


If the particle in on the other hand a _fermion_, then the action of the operator is given by:
\begin{align}
\hat{a}_i^\dagger |\cdots, n_i, \cdots\rangle = (-1)^{\sum_{j<i}n_j}(1-n_i)|\cdots, n_i+1, \cdots\rangle 
\end{align}

In both cases the number operator reads:
\begin{align}
\hat{n}_i &= \hat{a}_i^\dagger \hat{a}_i\\
\hat{n}_i |\cdots, n_i, \cdots\rangle &=n_i |\cdots, n_i, \cdots\rangle
\end{align}
This means that each orbit can be only occupied by a single fermion at most (known as Pauli exclusion).


In the very common approximation of a Hubbard model, we can now write down the Hamiltonian of the system as:
\begin{align}
\hat{H} = -J \sum_{\langle i,j\rangle}(\hat{a}_i^\dagger \hat{a}_j +\hat{a}_j^\dagger \hat{a}_i)+ \sum_i \mu_i \hat{n}_i + \frac{U}{2}\sum_i \hat{n}_i (\hat{n}_i+1)
\end{align}
The different ingredients are:
 - The hopping from one site to another with speed J
 - A potential energy on each site of strength $\mu_i$
 - An interaction between particles on a specific site of strength $U$
 
Importantly the Hamiltonian is conserving the total number of atoms $N = \sum_i n_i$, which means that we can look into systems of fixed atom number. _It would be very helpful to implement this part in a programmatic fashion, but for the moment we will make it real simple and then maybe fall back to tools like quspin_


# A system with two sites

Let us assume that we now have the smallest physical system of hopping particles. It consists of two sites, which we will call $0$ and $1$. We will now attempt to put particles into this system and let it evolve under the appropiate Hamiltonian. 

## Fermions

Let us start out with Fermions. 

- The situation is quite boring if no fermion is in the system. 
- Now assume that a single fermion is in the system, then only two states are allowed. The Fermion might be only on the left or the right and we are back to a problem of dimensionality $2$. 
- The situation is once again quite boring if we have two fermions in the system as the system is completely blocked now.

Let us write out the Hamiltonian and just see this. The basis of the system is the occupation numbers. Let us choose the following basis:
\begin{align}
|0\rangle = |0,0\rangle\\
|1\rangle = |1,0\rangle\\
|2\rangle = |0,1\rangle\\
|3\rangle = |1,1\rangle
\end{align}

We then have:
\begin{align}
\hat{a}_0^\dagger|0\rangle &= |1\rangle\\
\hat{a}_0^\dagger|1\rangle &= 0\\
\hat{a}_0^\dagger|2\rangle &= |3\rangle\\
\hat{a}_0^\dagger|3\rangle &= 0\\
\end{align}
and:
\begin{align}
\hat{a}_1^\dagger|0\rangle &= |2\rangle\\
\hat{a}_1^\dagger|1\rangle &= -|3\rangle\\
\hat{a}_1^\dagger|2\rangle &= 0\\
\hat{a}_1^\dagger|3\rangle &= 0\\
\end{align}

In [26]:
a0 = np.matrix([[0, 1, 0,0],[0, 0, 0,0],[0, 0, 0,1],[0, 0, 0,0]]);
a1 = np.matrix([[0, 0, 1,0],[0, 0, 0,-1],[0, 0, 0,0],[0, 0, 0,0]]);
a0dag = a0.T;
a1dag = a1.T;

n0 = a0dag.dot(a0)
n1 = a1dag.dot(a1)


Now we can build up the Hamiltonian

In [27]:
J = 1;
mu = np.array([1,1]);
U = 1;

HamTun = -J*(a0dag*a1+a1dag*a0)
HamSite = mu[0]*n0+mu[1]*n1;
HamInt = U/2*(n0*(n0-np.eye(4))+n1*(n1-np.eye(4)))

Ham = HamTun+HamSite+HamInt;
print(Ham)

[[ 0.  0.  0.  0.]
 [ 0.  1. -1.  0.]
 [ 0. -1.  1.  0.]
 [ 0.  0.  0.  2.]]


And we see now what we discussed already before. Let us just see how we would create a superposition state of a single particle on both sites. 

In [28]:
J = 1;t = np.pi/4
mu = 0*np.array([1,1]);
U = 0;

HamTun = -J*(a0dag*a1+a1dag*a0)
HamSite = mu[0]*n0+mu[1]*n1;
HamInt = U/2*(n0*(n0-np.eye(4))+n1*(n1-np.eye(4)))

Ham = HamTun+HamSite+HamInt;

psiI = np.array([0,1,0,0]).T;
U = expm(1j*Ham*t);
psiF = U.dot(psiI)
print(psiF)

[0.        +0.j         0.70710678+0.j         0.        -0.70710678j
 0.        +0.j        ]


## Bosons

Let us start out with Bosons. 

- The situation is quite boring if no boson is in the system. 
- Now assume that a single boson is in the system, then only two states are allowed. The boson might be only on the left or the right and we are back to a problem of dimensionality $2$. The problem is identical to the case of a single fermion. We know this limit as the limit of single particle physics.
- The situation is becoming really interesting once we have two bosons as the might even interact.

Let us write out the Hamiltonian and just see this. We will artifically cut of the basis later
The basis of the system is the occupation numbers. Let us choose the following basis:
\begin{align}
|0\rangle = |0,0\rangle\\
|1\rangle = |1,0\rangle\\
|2\rangle = |0,1\rangle\\
|3\rangle = |1,1\rangle\\
|4\rangle = |2,0\rangle\\
|5\rangle = |0,2\rangle\\
\end{align}

We then have:
\begin{align}
\hat{a}_0^\dagger|0\rangle &= |1\rangle\\
\hat{a}_0^\dagger|1\rangle &= \sqrt{2}|4\rangle\\
\hat{a}_0^\dagger|2\rangle &= |3\rangle
\end{align}
While we could now raise further, we will cut of the Hilbert space for occupation numbers of a total of 2 and write:
\begin{align}
\hat{a}_0^\dagger|3\rangle &= \hat{a}_0^\dagger|4\rangle = \hat{a}_0^\dagger|5\rangle = 0\\
\end{align}
For the second site we obtain:
\begin{align}
\hat{a}_1^\dagger|0\rangle &= |2\rangle\\
\hat{a}_1^\dagger|1\rangle &= |3\rangle\\
\hat{a}_1^\dagger|2\rangle &= \sqrt{2}|5\rangle\\
\hat{a}_1^\dagger|3\rangle &= \hat{a}_1^\dagger|4\rangle=\hat{a}_1^\dagger|5\rangle= 0\\
\end{align}

In [29]:
a0 = np.matrix([[0, 1, 0,0,0,0],[0, 0, 0,0, np.sqrt(2),0],[0, 0, 0,1,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0]]);
a1 = np.matrix([[0, 0, 1,0,0,0],[0, 0, 0,1,0,0],[0, 0, 0,0,0,np.sqrt(2)],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0],[0, 0, 0,0,0,0]]);
a0dag = a0.T;
a1dag = a1.T;

n0 = a0dag.dot(a0)
n1 = a1dag.dot(a1);
print(n0+n1)

[[0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 2.]]


So the first line describes no atoms in the system etc... Now we can build up the Hamiltonian

In [30]:
J = 1;
mu = 0*np.array([1,1]);
U = 0;

HamTun = -J*(a0dag*a1+a1dag*a0)
HamSite = mu[0]*n0+mu[1]*n1;
HamInt = U/2*(n0*(n0-np.eye(6))+n1*(n1-np.eye(6)))

Ham = HamTun+HamSite+HamInt;
print(Ham)

[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.          0.        ]
 [ 0.         -1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.41421356 -1.41421356]
 [ 0.          0.          0.         -1.41421356  0.          0.        ]
 [ 0.          0.          0.         -1.41421356  0.          0.        ]]


And we see now what we discussed already before.